In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
def build_generator(latent_dim):
    model = keras.Sequential()
    model.add(layers.Dense(256, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(784, activation='tanh'))
    model.add(layers.Reshape((28, 28, 1)))
    return model

In [4]:
def build_discriminator(img_shape):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=img_shape))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [5]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = keras.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [6]:
(X_train, _), (_, _) = keras.datasets.mnist.load_data()
X_train = X_train / 127.5 - 1.
X_train = np.expand_dims(X_train, axis=3)

11490434/11490434 [==============================] - 4s 0us/step


In [7]:
latent_dim = 100
img_shape = (28, 28, 1)
optimizer = keras.optimizers.Adam(0.0002, 0.5)
loss_fn = keras.losses.BinaryCrossentropy()

In [8]:
generator = build_generator(latent_dim)
discriminator = build_discriminator(img_shape)
gan = build_gan(generator, discriminator)

In [9]:
generator.compile(loss='binary_crossentropy', optimizer=optimizer)
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
gan.compile(loss='binary_crossentropy', optimizer=optimizer)

In [10]:
epochs = 500
batch_size = 128
steps_per_epoch = int(X_train.shape[0] / batch_size)

In [11]:
for epoch in range(epochs):
    for step in range(steps_per_epoch):
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)
        real_imgs = X_train[np.random.randint(0, X_train.shape[0], batch_size)]
        d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

2/4 [==============>...............] - ETA: 0s

In [ ]:
noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

    print(f"Epoch {epoch}/{epochs}, Discriminator Loss: {d_loss}, Generator Loss: {g_loss}")

In [ ]:
import matplotlib.pyplot as plt

if epoch % 100 == 0:
        noise = np.random.normal(0, 1, (10, latent_dim))
        gen_imgs = generator.predict(noise)
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(1, 10, figsize=(10, 1))
        for i in range(10):
            axs[i].imshow(gen_imgs[i, :, :, 0], cmap='gray')
            axs[i].axis('off')
        plt.savefig(f"generated_images/epoch_{epoch}.png")
        plt.close()


In [ ]:
generator.save('gan_model.h5')